In [81]:
import pandas as pd
from glob import glob


In [82]:
infos = glob('./info*')
df = pd.DataFrame(infos, columns = ['info_path'])
df['meta'] = df.info_path.apply(lambda x: pd.read_csv(x, index_col=0)['0'].to_dict())
df.meta.apply(pd.Series).sort_values('dataset')

,dataset,X_size,Met2Img
0,CRC,"(121, 23, 23, 1)",fillup-spb-gray
5,CRC,"(121, 23, 23, 3)",fillup-spb-jet
3,Cirrhosis,"(232, 24, 24, 3)",fillup-spb-jet
8,Cirrhosis,"(232, 24, 24, 1)",fillup-spb-gray
1,IBD,"(110, 22, 22, 3)",fillup-spb-jet
6,IBD,"(110, 22, 22, 1)",fillup-spb-gray
2,Obesity,"(253, 22, 22, 3)",fillup-spb-jet
7,Obesity,"(253, 22, 22, 1)",fillup-spb-gray
4,T2D,"(344, 24, 24, 3)",fillup-spb-jet
9,T2D,"(344, 24, 24, 1)",fillup-spb-gray


In [83]:
def _get_mean(x):
    l1 = pd.read_csv(x, index_col=0)
    return pd.DataFrame(l1).groupby('repeat_seed').mean().mean()

def _get_std(x):
    l1 = pd.read_csv(x, index_col=0)
    return pd.DataFrame(l1).groupby('repeat_seed').mean().std()


pfs = glob('./Met2Img*')
df = pd.DataFrame(pfs, columns = ['result_path'])


mean = df.result_path.apply(lambda x: _get_mean(x))
std = df.result_path.apply(lambda x: _get_std(x))

df = df.result_path.apply(lambda x:x.split('_')).apply(pd.Series)
df.columns = ['Method', 'dataset', 'config']


mean_stds = []
for x in mean.columns[:-1]:
    s = mean[x].round(3).astype(str) + '±' + std[x].round(3).astype(str)
    s.to_frame(name = x)
    mean_stds.append(s)
    
mean_stds = pd.concat(mean_stds, axis=1)

df = df.join(mean_stds)
df = df.sort_values('dataset')

In [84]:
df.config = df.config.apply(lambda x:x.replace('.csv', ''))

In [85]:
df.set_index(['Method', 'dataset', 'config'])

acc          auc          mcc  \
Method    dataset   config                                                   
./Met2Img CRC       fillup-spb-gray   0.673±0.01  0.796±0.013  0.286±0.025   
                    fillup-spb-jet   0.707±0.015  0.808±0.022    0.37±0.03   
          Cirrhosis fillup-spb-gray  0.842±0.008  0.922±0.006  0.696±0.017   
                    fillup-spb-jet    0.855±0.01  0.929±0.006    0.72±0.02   
          IBD       fillup-spb-jet     0.772±0.0  0.811±0.021      0.0±0.0   
                    fillup-spb-gray    0.772±0.0  0.847±0.026      0.0±0.0   
          Obesity   fillup-spb-jet     0.648±0.0   0.651±0.01      0.0±0.0   
                    fillup-spb-gray    0.648±0.0  0.643±0.009      0.0±0.0   
          T2D       fillup-spb-gray  0.634±0.016   0.714±0.01  0.281±0.034   
                    fillup-spb-jet   0.629±0.013  0.709±0.011  0.265±0.022   

                                            pres       recall           f1  
Method    dataset   config                                                  
./Met2Img CRC       fillup-spb-gray  0.674±0.027   0.673±0.01  0.613±0.014  
                    fillup-spb-jet   0.715±0.024  0.707±0.015  0.667±0.015  
          Cirrhosis fillup-spb-gray   0.854±0.01  0.842±0.008  0.841±0.008  
                    fillup-spb-jet    0.865±0.01   0.855±0.01   0.854±0.01  
          IBD       fillup-spb-jet     0.599±0.0    0.772±0.0    0.674±0.0  
                    fillup-spb-gray    0.599±0.0    0.772±0.0    0.674±0.0  
          Obesity   fillup-spb-jet     0.421±0.0    0.648±0.0     0.51±0.0  
                    fillup-spb-gray    0.421±0.0    0.648±0.0     0.51±0.0  
          T2D       fillup-spb-gray  0.648±0.018  0.634±0.016  0.627±0.016  
                    fillup-spb-jet    0.638±0.01  0.629±0.013  0.622±0.016